In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# -----------------------------
# 取得したいベースURL（クエリのpage番号だけ変える想定）
# -----------------------------
base_url = "https://j-president.net/Search?q=%E6%84%9B%E7%9F%A5%E7%9C%8C&pref=23&page={}"

# ページ範囲（例：1ページ目から3ページ目まで）
start_page = 1
end_page = 6

# 結果格納用
all_links = []

# -----------------------------
# ページを順に取得
# -----------------------------
headers = {"User-Agent": "Mozilla/5.0"}
for page in range(start_page, end_page + 1):
    url = base_url.format(page)
    print(f"▶ ページ取得中: {url}")

    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        print(f"❌ ページ取得失敗: {url}")
        continue

    soup = BeautifulSoup(res.text, "html.parser")

    # <ul class="list clearfix"> 内の <li> にある <a> をすべて取得
    ul = soup.find("ul", class_="list clearfix")
    if not ul:
        print("⚠️ 該当するulが見つかりませんでした。")
        continue

    for a_tag in ul.find_all("a", href=True):
        link = a_tag["href"]
        # 絶対URLでない場合もあるので一応補完
        if link.startswith("/"):
            link = "https://j-president.net" + link
        all_links.append(link)


df = pd.DataFrame(all_links, columns=["URL"])

# CSVに保存（UTF-8 BOM付きなら Excel でも文字化けしにくい）
df.to_csv("company_linkstv", index=False, encoding="utf-8-sig")
print("✅ company_links.csv に保存しました！")

▶ ページ取得中: https://j-president.net/Search?q=%E6%84%9B%E7%9F%A5%E7%9C%8C&pref=23&page=1
▶ ページ取得中: https://j-president.net/Search?q=%E6%84%9B%E7%9F%A5%E7%9C%8C&pref=23&page=2
▶ ページ取得中: https://j-president.net/Search?q=%E6%84%9B%E7%9F%A5%E7%9C%8C&pref=23&page=3
▶ ページ取得中: https://j-president.net/Search?q=%E6%84%9B%E7%9F%A5%E7%9C%8C&pref=23&page=4
▶ ページ取得中: https://j-president.net/Search?q=%E6%84%9B%E7%9F%A5%E7%9C%8C&pref=23&page=5
▶ ページ取得中: https://j-president.net/Search?q=%E6%84%9B%E7%9F%A5%E7%9C%8C&pref=23&page=6
✅ company_links.csv に保存しました！


In [10]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# 1. company_links.csv を読み込み
df_links = pd.read_csv("company_linkstv")  # カラム名が 'URL' などなら合わせてください
# 例: CSVに「URL」というカラムがある前提
link_list = df_links["URL"].dropna().tolist()

headers = {"User-Agent": "Mozilla/5.0"}
results = []

# 2. 各詳細ページを巡回して情報を取得
for link in link_list:
    print(f"▶ 詳細ページ取得中: {link}")
    try:
        res = requests.get(link, headers=headers, timeout=10)
        if res.status_code != 200:
            print(f"❌ 取得失敗: {link}")
            continue

        soup = BeautifulSoup(res.text, "html.parser")

        company_name = ""
        company_hps = []

        for dl in soup.find_all("dl"):
            dt = dl.find("dt")
            dd = dl.find("dd")
            if not dt or not dd:
                continue
            dt_text = dt.get_text(strip=True)
            if "会社名" in dt_text:
                company_name = dd.get_text(strip=True)
            if "企業HP" in dt_text:
                for a in dd.find_all("a", href=True):
                    company_hps.append(a["href"])

        results.append({
            "会社名": company_name,
            "企業HP": ",".join(company_hps),
            "詳細URL": link
        })

    except Exception as e:
        print(f"⚠ エラー発生: {link} -> {e}")

    time.sleep(1)  # サイト負荷軽減

# 3. 結果をDataFrame化
df_result = pd.DataFrame(results)
print(df_result)

# 必要に応じてCSV保存
df_result.to_csv("company_info_from_linkstv.csv", index=False, encoding="utf-8-sig")


▶ 詳細ページ取得中: http://j-president.net/aichi/owaririkuun/
▶ 詳細ページ取得中: http://j-president.net/aichi/suncare-japan/
▶ 詳細ページ取得中: http://j-president.net/aichi/ddaisei/
▶ 詳細ページ取得中: http://j-president.net/aichi/cosmos-gr/
▶ 詳細ページ取得中: http://j-president.net/aichi/sugimotogumi/
▶ 詳細ページ取得中: http://j-president.net/aichi/monogatari/
▶ 詳細ページ取得中: http://j-president.net/aichi/tokai-sanyu-technos/
▶ 詳細ページ取得中: http://j-president.net/aichi/hirate-giken/
▶ 詳細ページ取得中: http://j-president.net/aichi/bebrain/
▶ 詳細ページ取得中: http://j-president.net/aichi/daitoku-iv/
▶ 詳細ページ取得中: http://j-president.net/aichi/kusunoki-dieholder/
▶ 詳細ページ取得中: http://j-president.net/aichi/rinnai/
▶ 詳細ページ取得中: http://j-president.net/aichi/hidakafamily/
▶ 詳細ページ取得中: http://j-president.net/aichi/chubudoboku/
▶ 詳細ページ取得中: http://j-president.net/aichi/towa-jushi/
▶ 詳細ページ取得中: http://j-president.net/aichi/akane-fukushi-service/
▶ 詳細ページ取得中: http://j-president.net/aichi/hamajimabousaisystem/
▶ 詳細ページ取得中: http://j-president.net/aichi/succeed-bp/
▶ 詳細ページ取